In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.stats import pearsonr
from scipy.stats import norm

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

## Loading Data

In [2]:
with open('../data/X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('../data/X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('../data/y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('../data/y_test.npy', 'rb') as f:
    y_test = np.load(f)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13972, 29) (6872, 29) (13972,) (6872,)


In [15]:
with open('X_train2.npy', 'rb') as f:
    X_train2 = np.load(f)
with open('X_test2.npy', 'rb') as f:
    X_test2 = np.load(f)
with open('X_train3.npy', 'rb') as f:
    X_train3 = np.load(f)
with open('X_test3.npy', 'rb') as f:
    X_test3 = np.load(f)

In [3]:
columns = ['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType',
 'Month_Aug',
 'Month_Dec',
 'Month_Feb',
 'Month_Jul',
 'Month_June',
 'Month_Mar',
 'Month_May',
 'Month_Nov',
 'Month_Oct',
 'Month_Sep',
 'VisitorType_New_Visitor',
 'VisitorType_Other',
 'VisitorType_Returning_Visitor',
 'Weekend_False',
 'Weekend_True',
 'Revenue']

feature_names = columns
feature_names.remove("Revenue")

In [4]:
X_train = pd.DataFrame(X_train, columns=feature_names)
X_train = X_train.drop("Weekend_False",axis=1)
X_test = pd.DataFrame(X_test, columns=feature_names)
X_test = X_test.drop("Weekend_False",axis=1)

In [5]:
X_DF = pd.concat([X_train,X_test])
y = np.concatenate((y_train,y_test))

## Tuning Hyperparameters

In [21]:
model = AdaBoostClassifier()

In [7]:
param_dist = {
    "n_estimators":   list(np.linspace(200, 1600, 8, dtype=int)),
    "learning_rate": [0.15, 0.1, 0.05, 0.01, 0.005, 0.001]
}
# Random search of parameters, using cross validation, use all cores
#model_rand = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=0, n_jobs=-1)

In [20]:
def get_best_params(model,param_dist,X,y):
    model_rand = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=0, n_jobs=-1)
    model_rand.fit(X,y)
    best_params = model_rand.best_params_
    return best_params

In [27]:
best_params = get_best_params(model,param_dist,X_train,y_train)
print(best_params)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/Users/erinszeto/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 400, 'learning_rate': 0.05}


## Fitting the Model

In [10]:
model_best = AdaBoostClassifier(**best_params)
model_best.fit(X_train,y_train) 

AdaBoostClassifier(learning_rate=0.05, n_estimators=400)

In [11]:
score = cross_val_score(model_best, X_train, y_train)
print(np.round(score.mean(), 4))

0.8965


In [12]:
def get_performance(model,X,y):
    y_pred = model.predict(X)
    target_names = ['0: No Revenue', '1: Revenue']
    report = classification_report(y_test, y_pred, target_names=target_names)
    
    print(report)
    print("Confusion Matrix:\n",confusion_matrix(y,y_pred))
    print("Accuracy: ",accuracy_score(y,y_pred))
    print("ROC-AUC: ",roc_auc_score(y,model.predict_proba(X)[:,1]))

In [13]:
get_performance(model_best,X_test,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.92      0.91      0.92      3436
   1: Revenue       0.91      0.92      0.92      3436

     accuracy                           0.92      6872
    macro avg       0.92      0.92      0.92      6872
 weighted avg       0.92      0.92      0.92      6872

Confusion Matrix:
 [[3133  303]
 [ 271 3165]]
Accuracy:  0.9164726426076834
ROC-AUC:  0.9747612589292853


## Model with Feature Engineering

In [24]:
model2 = AdaBoostClassifier(**best_params)
model2.fit(X_train2, y_train)

AdaBoostClassifier(learning_rate=0.005, n_estimators=1400)

In [26]:
get_performance(model2,X_test2,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.94      0.88      0.91      3436
   1: Revenue       0.89      0.94      0.92      3436

     accuracy                           0.91      6872
    macro avg       0.92      0.91      0.91      6872
 weighted avg       0.92      0.91      0.91      6872

Confusion Matrix:
 [[3039  397]
 [ 190 3246]]
Accuracy:  0.9145809080325961
ROC-AUC:  0.9680817858841737


In [28]:
model2 = AdaBoostClassifier(**best_params)
model2.fit(X_train2, y_train)
get_performance(model2,X_test2,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.95      0.89      0.92      3436
   1: Revenue       0.90      0.95      0.92      3436

     accuracy                           0.92      6872
    macro avg       0.92      0.92      0.92      6872
 weighted avg       0.92      0.92      0.92      6872

Confusion Matrix:
 [[3067  369]
 [ 177 3259]]
Accuracy:  0.920547147846333
ROC-AUC:  0.9794721303299584


## Model with Feature Selection and Feature Engineering

In [29]:
model3 = AdaBoostClassifier(**best_params)
model3.fit(X_train3, y_train)

AdaBoostClassifier(learning_rate=0.05, n_estimators=400)

In [30]:
get_performance(model3,X_test3,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.95      0.89      0.92      3436
   1: Revenue       0.90      0.95      0.92      3436

     accuracy                           0.92      6872
    macro avg       0.92      0.92      0.92      6872
 weighted avg       0.92      0.92      0.92      6872

Confusion Matrix:
 [[3064  372]
 [ 177 3259]]
Accuracy:  0.9201105937136205
ROC-AUC:  0.9794070368392737
